# Multiscale MILP with Materials and Emission

This is a continuation of 'One Location, Multiple Temporal Scales, Multiple Operations, Mixed Integer Linear Programming Example' [Example 2]. 

We will be adding the following considerations: 

1. material for the establishment of operations 
2. global warming potential induced as a sum of our decisions and choices 

## See Example 2

In [1]:
from energia import *

m = Model('example3')
m.q = Periods()
m.y = 4 * m.q

In [2]:
m.usd = Currency()
m.declare(Resource, ['power', 'wind', 'solar'])
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49

m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10
m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

m.lii = Storage()
m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432
m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.charge.operate <= 1
m.lii.discharge.capacity <= 100
m.lii.discharge.operate <= 1

2025-10-23 13:29:35,039 [INFO] Balance for solar in (l0, q): initializing
2025-10-23 13:29:35,040 [INFO] ✔ Completed in 0.00021266937255859375 seconds
2025-10-23 13:29:35,040 [INFO] Binding consume in domain (solar, l0, q)
2025-10-23 13:29:35,041 [INFO] ✔ Completed in 0.0001575946807861328 seconds
2025-10-23 13:29:35,041 [INFO] Balance for wind in (l0, y): initializing
2025-10-23 13:29:35,042 [INFO] ✔ Completed in 0.00013518333435058594 seconds
2025-10-23 13:29:35,042 [INFO] Binding consume in domain (wind, l0, y)
2025-10-23 13:29:35,043 [INFO] ✔ Completed in 9.059906005859375e-05 seconds
2025-10-23 13:29:35,044 [INFO] Balance for power in (l0, q): initializing
2025-10-23 13:29:35,044 [INFO] ✔ Completed in 0.00015497207641601562 seconds
2025-10-23 13:29:35,045 [INFO] Binding release in domain (power, l0, q)
2025-10-23 13:29:35,045 [INFO] ✔ Completed in 0.00017499923706054688 seconds
2025-10-23 13:29:35,046 [INFO] Binding capacity in domain (wf, l0, y)
2025-10-23 13:29:35,046 [INFO] ✔ C

aaa η(lii.discharge, None)


## Indicator Stream 

Indicators scale the impact of some decision or flow and project it only a common dimension allowing the calculation of overall impact as a single metric value

In this case, we consider global warming potential (GWP).

In [3]:
m.gwp = Environ()

Now say that we want to consider the impact of setting up operations (specific to the act of construction)

These can be provided as calculations 

In [4]:
m.wf.capacity[m.gwp.emit] == 1000
m.pv.capacity[m.gwp.emit] == 2000
m.lii.capacity[m.gwp.emit] == 3000
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note that impact streams are mapped across scales but not balanced, unless a negative (relatively) impact is also assessed! In the case of emission, this would be abatement.

## Material Stream

Note that all the objects found in energia.components.commodity.misc are still resources (Land, Material, etc.)

This will allow for the modeling of an expansive type of processes.

Examples:

1. land clearance to transform agricultural land into industrial, or land remediation 
2. resource intense material recycling 
3. some resource flowing being treated as emission flows 

The possibilities are vast... The advanced modeler may prefer to use Resource() for everything [see Resource Task Network (RTN) Framework]

In the following example, we consider a limit to cement consumption, as also an expense and gwp impact associated with it

In [5]:
m.cement = Material()
m.cement.consume <= 1000000
m.cement.consume[m.usd.spend] == 17
m.cement.consume[m.gwp.emit] == 0.9

2025-10-23 13:29:35,082 [INFO] Balance for cement in (l0, y): initializing
2025-10-23 13:29:35,083 [INFO] ✔ Completed in 0.00020003318786621094 seconds
2025-10-23 13:29:35,084 [INFO] Binding consume in domain (cement, l0, y)
2025-10-23 13:29:35,084 [INFO] ✔ Completed in 9.274482727050781e-05 seconds


Next, we provide details of the use of cement across all operations 

In [6]:
m.wf.capacity[m.cement.use] == 400
m.pv.capacity[m.cement.use] == 560
m.lii.capacity[m.cement.use] == 300

2025-10-23 13:29:35,091 [INFO] Balance for cement in (l0, y): adding use(cement, l0, y, capacity, wf)
2025-10-23 13:29:35,091 [INFO] ✔ Completed in 0.0002818107604980469 seconds
2025-10-23 13:29:35,092 [INFO] Balance for cement in (l0, y): adding use(cement, l0, y, capacity, pv)
2025-10-23 13:29:35,093 [INFO] ✔ Completed in 0.00012731552124023438 seconds
2025-10-23 13:29:35,094 [INFO] Balance for cement in (l0, y): adding use(cement, l0, y, invcapacity, lii.stored)
2025-10-23 13:29:35,095 [INFO] ✔ Completed in 0.0001220703125 seconds


Material use is summed up across all operations, and adheres to an upper bound in terms of consumption. 

In [7]:
m.cement.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

GWP impact is now the sum of impact from material use as well as constuction 

In [8]:
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Locate Operations

In [9]:
m.network.locate(m.wf, m.pv, m.lii)

2025-10-23 13:29:35,119 [INFO] Balance for power in (l0, q): adding produce(power, l0, q, operate, wf)
2025-10-23 13:29:35,120 [INFO] ✔ Completed in 0.0002014636993408203 seconds
2025-10-23 13:29:35,121 [INFO] Balance for wind in (l0, y): adding expend(wind, l0, y, operate, wf)
2025-10-23 13:29:35,122 [INFO] ✔ Completed in 0.0001354217529296875 seconds
2025-10-23 13:29:35,123 [INFO] Balance for power in (l0, q): adding produce(power, l0, q, operate, pv)
2025-10-23 13:29:35,124 [INFO] ✔ Completed in 0.0001819133758544922 seconds
2025-10-23 13:29:35,125 [INFO] Balance for solar in (l0, q): adding expend(solar, l0, q, operate, pv)
2025-10-23 13:29:35,125 [INFO] ✔ Completed in 0.00017523765563964844 seconds
2025-10-23 13:29:35,126 [INFO] Assuming  lii.stored inventory capacity is unbounded in (l0, y)
2025-10-23 13:29:35,126 [INFO] Assuming inventory of lii.stored is unbounded in (l0, y)
2025-10-23 13:29:35,127 [INFO] Assuming inventory of lii.stored is bound by inventory capacity in (l0, q

## Optimization 

Let us minimize GWP this time. Note that m.usd.spend.opt() can still be used!

In [10]:
m.gwp.emit.opt()

2025-10-23 13:29:35,147 [INFO] Mapping emit: (gwp, l0, y, capacity, wf) → (gwp, l0, y)
2025-10-23 13:29:35,148 [INFO] ✔ Completed in 0.00015854835510253906 seconds
2025-10-23 13:29:35,149 [INFO] Mapping emit: (gwp, l0, y, capacity, pv) → (gwp, l0, y)
2025-10-23 13:29:35,149 [INFO] ✔ Completed in 0.0001430511474609375 seconds
2025-10-23 13:29:35,150 [INFO] Mapping emit: (gwp, l0, y, invcapacity, lii.stored) → (gwp, l0, y)
2025-10-23 13:29:35,150 [INFO] ✔ Completed in 0.00012493133544921875 seconds
2025-10-23 13:29:35,151 [INFO] Mapping emit: (gwp, l0, y, consume, cement) → (gwp, l0, y)
2025-10-23 13:29:35,151 [INFO] ✔ Completed in 0.00013113021850585938 seconds
2025-10-23 13:29:35,152 [INFO] Generating Program(example3).mps
2025-10-23 13:29:35,156 [INFO] Creating gurobi model for Program(example3)


Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-01
Read MPS format model from file Program(example3).mps
Reading time = 0.00 seconds
PROGRAM(EXAMPLE3): 95 rows, 87 columns, 217 nonzeros


2025-10-23 13:29:35,162 [INFO] Optimizing Program(example3) using gurobi


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 95 rows, 87 columns and 217 nonzeros
Model fingerprint: 0x39a95d48
Variable types: 84 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 1e+06]
Presolve removed 83 rows and 76 columns
Presolve time: 0.00s
Presolved: 12 rows, 11 columns, 38 nonzeros
Variable types: 11 continuous, 0 integer (0 binary)

Root relaxation: objective 4.752148e+05, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    475214.81481 475214.815  0.0

2025-10-23 13:29:35,175 [INFO] Solution found. Use .output() to display it
2025-10-23 13:29:35,175 [INFO] Creating Solution object, check.solution


In [11]:
len(m.variables)

87

In [12]:
m.show()

# Mathematical Program for Program(example3)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###  Function Sets

<IPython.core.display.Math object>

In [13]:
m.output(False)

# Solution for Program(example3)

<br><br>

## Objective

<IPython.core.display.Math object>

<br><br>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>